# Sesiunea 6: Complex Data Sources + AI pentru Data Science

## 📋 Obiective

**Partea 1: Complex Data Sources (45 min)**
- Lucru cu Excel (multiple sheets)
- Merge și concatenare datasets
- Export în formate complexe

**Partea 2: AI Integration (45 min)**
- Folosirea ChatGPT pentru cod pandas
- Folosirea Claude pentru debugging
- Best practices și limitări

**Durata:** 1.5 ore

**Finalizăm proiectul EU-SILC - Sesiunea 6/6** 🎉

# PARTEA 1: COMPLEX DATA SOURCES

## 1.1 Setup

In [1]:
import pandas as pd
import numpy as np

print("✅ Libraries loaded!")

✅ Libraries loaded!


## 1.2 Working with Excel - Multiple Sheets

Excel files often contain multiple sheets with related data. Let's see how to work with them.

In [2]:
# First, let's create an example Excel file with multiple sheets
# (In real scenarios, you'd receive this file)

# Load our clean data
df = pd.read_csv('../datasets/eusilc_clean.csv')

# Create a metadata dictionary (variable descriptions)
metadata = pd.DataFrame({
    'Variable': ['HB010', 'HB020', 'HB030', 'HY010', 'HY020', 'HY022', 'HY023',
                 'HH030', 'HX050', 'HS011', 'HS021', 'HS040',
                 'HS110', 'HS120', 'HS130', 'HS140', 'HS150', 'HS160', 'HH010'],
    'Description': [
        'Survey year',
        'Country',
        'Household ID',
        'Total household gross income',
        'Total disposable income from work',
        'Income from self-employment',
        'Income from farming',
        'Household size',
        'Equivalised household size',
        'Type of dwelling',
        'Tenure status',
        'Number of rooms',
        'Bath or shower in dwelling',
        'Financial burden of housing costs',
        'Arrears on payments',
        'Capacity to face unexpected expenses',
        'Capacity to afford one week holiday',
        'Capacity to afford meal with meat/fish',
        'Degree of urbanisation'
    ],
    'Type': ['int', 'str', 'int', 'float', 'float', 'float', 'float',
             'int', 'float', 'int', 'int', 'int',
             'int', 'int', 'int', 'int', 'int', 'int', 'int']
})

# Create summary statistics
summary_stats = df[['HY010', 'HY020', 'HY022', 'HY023', 'HH030']].describe()

print("✅ Example data prepared")
print(f"\nMetadata: {len(metadata)} variables documented")
metadata.head()

KeyError: "None of [Index(['HY010', 'HY020', 'HY022', 'HY023', 'HH030'], dtype='object')] are in the [columns]"

### Creating Multi-Sheet Excel

In [ ]:
# Create Excel file with multiple sheets
excel_path = '../datasets/eusilc_with_metadata.xlsx'

with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
    # Sheet 1: First 100 rows of data (sample)
    df.head(100).to_excel(writer, sheet_name='Data_Sample', index=False)
    
    # Sheet 2: Metadata
    metadata.to_excel(writer, sheet_name='Variable_Descriptions', index=False)
    
    # Sheet 3: Summary statistics
    summary_stats.to_excel(writer, sheet_name='Summary_Statistics')

print(f"✅ Excel file created: {excel_path}")
print("\nSheets created:")
print("  1. Data_Sample - First 100 households")
print("  2. Variable_Descriptions - Metadata")
print("  3. Summary_Statistics - Descriptive stats")

### Reading Multi-Sheet Excel

In [ ]:
# Method 1: Read specific sheet
df_data = pd.read_excel(excel_path, sheet_name='Data_Sample')
print("Method 1: Read specific sheet")
print(f"Data shape: {df_data.shape}")

# Method 2: Read all sheets into dictionary
all_sheets = pd.read_excel(excel_path, sheet_name=None)
print(f"\nMethod 2: Read all sheets")
print(f"Sheets found: {list(all_sheets.keys())}")

# Method 3: Using ExcelFile for efficiency
with pd.ExcelFile(excel_path) as xls:
    print(f"\nMethod 3: ExcelFile object")
    print(f"Sheet names: {xls.sheet_names}")
    
    # Read specific sheet
    metadata_df = pd.read_excel(xls, 'Variable_Descriptions')
    print(f"\nMetadata loaded: {len(metadata_df)} variables")

## 1.3 Merging Datasets

Often you need to combine data from multiple sources.

In [ ]:
# Example: Merge main data with metadata
# Create a small sample for demonstration
df_sample = df.head(5)[['HB030', 'HY010', 'HH030']]

print("Original data (sample):")
print(df_sample)
print("\nMetadata:")
print(metadata[['Variable', 'Description']].head())

In [ ]:
# For merging, we need to reshape data to long format
df_long = df_sample.melt(id_vars=['HB030'], var_name='Variable', value_name='Value')

print("Data in long format:")
print(df_long.head(10))

# Now merge with metadata
df_with_metadata = df_long.merge(metadata[['Variable', 'Description']], 
                                   on='Variable', 
                                   how='left')

print("\nMerged data:")
print(df_with_metadata.head(10))

print("\n✅ Merge types:")
print("  - inner: only matching rows")
print("  - left: all from left, matching from right")
print("  - right: all from right, matching from left")
print("  - outer: all rows from both")

## 1.4 Concatenating Datasets

Concatenation stacks datasets vertically or horizontally.

In [ ]:
# Vertical concatenation (stacking rows)
df1 = df.head(3)
df2 = df.tail(3)

df_concat_vertical = pd.concat([df1, df2], axis=0, ignore_index=True)

print("Vertical concatenation (axis=0):")
print(f"df1 shape: {df1.shape}")
print(f"df2 shape: {df2.shape}")
print(f"Result shape: {df_concat_vertical.shape}")
print(f"\nCombined data:")
print(df_concat_vertical[['HB030', 'HY010']])

In [ ]:
# Horizontal concatenation (adding columns)
df_income = df[['HB030', 'HY010', 'HY020']].head(5)
df_housing = df[['HB030', 'HS011', 'HS040']].head(5)

# Remove HB030 from second to avoid duplication
df_housing_no_id = df_housing.drop('HB030', axis=1)

df_concat_horizontal = pd.concat([df_income, df_housing_no_id], axis=1)

print("\nHorizontal concatenation (axis=1):")
print(df_concat_horizontal)

## 1.5 Practical Example: Create Analysis Report

Let's combine everything to create a comprehensive Excel report.

In [ ]:
# Create income quintiles for report
df['income_quintile'] = pd.qcut(df['HY010'], q=5, 
                                  labels=['Q1 (Poorest)', 'Q2', 'Q3', 'Q4', 'Q5 (Richest)'])

# Create various analysis tables
quintile_summary = df.groupby('income_quintile')['HY010'].agg([
    ('Count', 'count'),
    ('Mean', 'mean'),
    ('Median', 'median'),
    ('Min', 'min'),
    ('Max', 'max')
]).round(2)

# Urban-rural comparison
urbanization_labels = {1: 'Urban', 2: 'Intermediate', 3: 'Rural'}
df['urbanization'] = df['HH010'].map(urbanization_labels)
urban_summary = df.groupby('urbanization')['HY010'].agg(['count', 'mean', 'median']).round(2)

# Create comprehensive Excel report
report_path = '../datasets/eusilc_analysis_report.xlsx'

with pd.ExcelWriter(report_path, engine='openpyxl') as writer:
    # Overview
    overview = pd.DataFrame({
        'Metric': ['Total Households', 'Variables', 'Survey Year', 'Country'],
        'Value': [len(df), len(df.columns), 2013, 'Estonia']
    })
    overview.to_excel(writer, sheet_name='Overview', index=False)
    
    # Income quintiles
    quintile_summary.to_excel(writer, sheet_name='Income_Quintiles')
    
    # Urban-Rural
    urban_summary.to_excel(writer, sheet_name='Urban_Rural_Comparison')
    
    # Variable metadata
    metadata.to_excel(writer, sheet_name='Metadata', index=False)
    
    # Full summary statistics
    df.describe().to_excel(writer, sheet_name='Summary_Statistics')

print(f"✅ Comprehensive report created: {report_path}")
print("\nReport contains:")
print("  1. Overview - Dataset summary")
print("  2. Income_Quintiles - Q1-Q5 analysis")
print("  3. Urban_Rural_Comparison - Geographic differences")
print("  4. Metadata - Variable descriptions")
print("  5. Summary_Statistics - Descriptive stats")

---

# PARTEA 2: AI PENTRU DATA SCIENCE

## 2.1 Introducere: De ce AI pentru Data Science?

**AI tools (ChatGPT, Claude) pot ajuta la:**
- ✅ Generare cod pandas rapid
- ✅ Debugging (găsirea erorilor)
- ✅ Explicare cod complex
- ✅ Sugestii de analiză
- ✅ Documentare cod

**Dar ATENȚIE:**
- ⚠️ AI face erori ("hallucinations")
- ⚠️ Trebuie să verificați codul
- ⚠️ Trebuie să înțelegeți ce face
- ⚠️ Nu întotdeauna e cel mai eficient cod

## 2.2 ChatGPT pentru Generare Cod

### Exemplu 1: Cerere simplă

In [ ]:
# PROMPT pentru ChatGPT:
# "Write pandas code to calculate income quintiles from a column named 'HY010'"

# RĂSPUNS ChatGPT (exemplu):
"""
df['income_quintile'] = pd.qcut(df['HY010'], q=5, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5'])
"""

# Verificăm:
print("✅ Cod generat de AI:")
print("df['income_quintile'] = pd.qcut(df['HY010'], q=5, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5'])")
print("\n💡 This is correct and works!")

### Exemplu 2: Cerere complexă

In [ ]:
# PROMPT pentru ChatGPT:
"""
I have a pandas DataFrame with columns HY010 (income), HH030 (household size), and HH010 (urbanization: 1=urban, 2=intermediate, 3=rural).
Write code to:
1. Create income quintiles
2. Calculate mean income per quintile and urbanization category
3. Create a pivot table showing this
"""

# RĂSPUNS ChatGPT (exemplu):
print("✅ AI-generated code:")
print("""
# Create quintiles
df['quintile'] = pd.qcut(df['HY010'], q=5, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5'])

# Create pivot table
pivot = df.pivot_table(
    values='HY010',
    index='quintile',
    columns='HH010',
    aggfunc='mean'
)

print(pivot)
""")

# Let's test it:
df_test = df.copy()
df_test['quintile'] = pd.qcut(df_test['HY010'], q=5, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5'])

pivot = df_test.pivot_table(
    values='HY010',
    index='quintile',
    columns='HH010',
    aggfunc='mean'
)

print("\n💡 AI code works! Pivot table:")
print(pivot.round(2))

## 2.3 Claude pentru Debugging

### Exemplu: Error fixing

In [ ]:
# Cod cu eroare (intentional)
print("❌ Cod cu eroare:")
print("""
# Acest cod va da eroare:
result = df.groupby('income_quintile')['HY010'].agg(['mean', 'median', 'std'])
result.sort_values('mean', ascending=False)
""")

print("\n⚠️ ERROR: 'mean' is not in index")
print("\nPROMPT pentru Claude:")
print("""
I'm getting this error:
'mean' is not in index

My code:
result = df.groupby('income_quintile')['HY010'].agg(['mean', 'median', 'std'])
result.sort_values('mean', ascending=False)

How do I fix it?
""")

print("\n✅ Claude's response:")
print("""
The issue is that 'mean' is a column name in your result, not an index.
You need to use brackets or specify axis:

result.sort_values(('mean',), ascending=False)
# OR
result.sort_values('mean', ascending=False, axis=0)
# OR better:
result.sort_values(by='mean', ascending=False)
""")

# Test the fix:
result = df.groupby('income_quintile')['HY010'].agg(['mean', 'median', 'std'])
result_sorted = result.sort_values(by='mean', ascending=False)
print("\n💡 Fixed code works:")
print(result_sorted.round(2))

## 2.4 Best Practices pentru AI Usage

### ✅ DO (Faceți):

1. **Use AI for starting points**
   - "Write a function to calculate Gini coefficient"
   - "Create a heatmap showing correlation between variables"

2. **Use AI for explaining code**
   - "Explain what this code does: [paste code]"
   - "What does pd.qcut() do differently from pd.cut()?"

3. **Use AI for documentation**
   - "Write a docstring for this function"
   - "Add comments to this code"

4. **Use AI for alternatives**
   - "Is there a faster way to do this?"
   - "Can this be done with vectorization instead of a loop?"

### ❌ DON'T (Nu faceți):

1. **Don't trust blindly**
   - Always test AI code
   - Verify statistical claims
   - Check for errors

2. **Don't use for critical decisions without validation**
   - "Is this statistically significant?" → Verify yourself
   - "What's the p-value?" → Calculate and check

3. **Don't copy-paste without understanding**
   - Read the code line by line
   - Understand each function
   - Know why it works

4. **Don't rely on AI for data cleaning decisions**
   - "Should I drop or impute?" → Depends on YOUR data and context
   - AI doesn't know your domain

## 2.5 Practical Example: Using AI to Enhance Our Analysis

In [ ]:
# PROMPT pentru ChatGPT:
"""
I have EU-SILC income data with quintiles already calculated.
Write a function that generates a summary report including:
- Income share per quintile (% of total income)
- Q5/Q1 ratio
- Mean and median per quintile
"""

# AI-generated function:
def generate_inequality_report(df, income_col='HY010', quintile_col='income_quintile'):
    """
    Generate inequality analysis report.
    
    Parameters:
    -----------
    df : DataFrame
        Data with income and quintile columns
    income_col : str
        Name of income column
    quintile_col : str
        Name of quintile column
        
    Returns:
    --------
    report : dict
        Dictionary with inequality metrics
    """
    # Calculate total income per quintile
    total_by_quintile = df.groupby(quintile_col)[income_col].sum()
    total_income = df[income_col].sum()
    
    # Income share
    income_share = (total_by_quintile / total_income * 100).round(2)
    
    # Mean and median
    stats = df.groupby(quintile_col)[income_col].agg(['mean', 'median']).round(2)
    
    # Q5/Q1 ratio
    q5_mean = stats.loc['Q5 (Richest)', 'mean']
    q1_mean = stats.loc['Q1 (Poorest)', 'mean']
    ratio = round(q5_mean / q1_mean, 2)
    
    report = {
        'income_share': income_share,
        'statistics': stats,
        'q5_q1_ratio': ratio
    }
    
    return report

# Test the function
report = generate_inequality_report(df)

print("📊 INEQUALITY REPORT (AI-generated function)")
print("=" * 60)
print("\nIncome Share (% of total):")
print(report['income_share'])
print("\nStatistics per Quintile:")
print(report['statistics'])
print(f"\nQ5/Q1 Ratio: {report['q5_q1_ratio']}x")
print("\n💡 AI helped create this analysis function in seconds!")

## 2.6 Limitări și Precauții

### Exemple de erori AI:

**1. Hallucinations (funcții inexistente):**

In [ ]:
# ❌ AI might suggest:
# df.calculate_gini()  # Această funcție NU EXISTĂ în pandas!

# ✅ Trebuie să implementați voi:
def gini_coefficient(x):
    """Calculate Gini coefficient (correctly)"""
    # Implementation here
    pass

print("⚠️ AI sometimes invents functions that don't exist!")
print("Always check pandas documentation.")

**2. Outdated syntax:**

In [ ]:
# ❌ AI might suggest old pandas syntax:
# df.ix[0]  # Deprecated!

# ✅ Modern pandas:
# df.iloc[0]  # Correct

print("⚠️ AI training data might be old.")
print("Check pandas version and use current syntax.")

**3. Statistical errors:**

In [ ]:
# ⚠️ AI might give wrong statistical advice:
# "Use mean to impute because it's unbiased"
# → Actually, median is often better for skewed distributions!

print("⚠️ Verify statistical claims!")
print("AI doesn't understand your data distribution.")

## Recapitulare Sesiunea 6

### ✅ PARTEA 1: Complex Data
- Excel multi-sheet: create, read, manipulate
- Merge: combining datasets by key
- Concat: stacking vertically/horizontally
- Export professional reports

### ✅ PARTEA 2: AI Integration
- ChatGPT for code generation
- Claude for debugging
- Best practices (DO/DON'T)
- Limitations and verification

### 🎓 Key Takeaways:

**Complex Data:**
1. Excel is powerful for reports (multiple sheets)
2. Merge when you have related data (common key)
3. Concat when you have similar structure
4. Always validate after combining!

**AI Usage:**
1. ✅ AI is a TOOL, not a replacement for thinking
2. ✅ Use for starting points and efficiency
3. ✅ Always verify and understand
4. ✅ Learn from AI code, don't just copy
5. ❌ Never trust blindly
6. ❌ Don't use for critical decisions without validation

---

## 🎉 CURS 2 COMPLET!

**Ce ați învățat în 6 sesiuni:**

1. **Sesiunea 1:** Pandas basics → DataFrames
2. **Sesiunea 2:** Data exploration → identify problems
3. **Sesiunea 3:** Data cleaning → create clean dataset
4. **Sesiunea 4:** EDA → understand distributions, inequality
5. **Sesiunea 5:** Visualization → professional charts
6. **Sesiunea 6:** Advanced → Excel, merge, AI

**Skills acquired:**
- ✅ pandas mastery
- ✅ Data cleaning strategies
- ✅ Statistical analysis
- ✅ Professional visualization
- ✅ Complex data handling
- ✅ AI-assisted coding

**You can now:**
- 📊 Analyze real survey data (EU-SILC, BNS, etc.)
- 🧹 Clean messy data professionally
- 📈 Create income inequality analyses
- 📉 Build dashboards and reports
- 🤖 Use AI to boost productivity
- 💼 Work as Data Analyst!

---

### 📚 Resources pentru continuare:

- [Pandas Documentation](https://pandas.pydata.org/docs/)
- [Kaggle Learn](https://www.kaggle.com/learn)
- [Real Python](https://realpython.com/)
- [ChatGPT](https://chat.openai.com/)
- [Claude](https://claude.ai/)

### 🚀 Next steps:

1. **Complete Homework 2** - practice everything
2. **Find your own dataset** - apply skills
3. **Build portfolio** - GitHub, showcase work
4. **Keep learning** - machine learning, time series, GIS
5. **Join community** - Python Moldova, data science groups

---

**🎉 Congratulations on completing Course 2! You're now a data scientist! 🎉**

*Happy analyzing! 📊🐍🚀*